# Punto 10

## Consigna

¿Cuáles son las diferencias entre `QDA_Chol1`, `QDA_Chol2` y `QDA_Chol3`?

In [8]:
import numpy as np
import numpy.linalg as LA
from scipy.linalg import cholesky, solve_triangular
from scipy.linalg.lapack import dtrtri

In [9]:
class BaseBayesianClassifier:
  def __init__(self):
    pass

  def _estimate_a_priori(self, y):
    a_priori = np.bincount(y.flatten().astype(int)) / y.size
    # Q3: para que sirve bincount?
    return np.log(a_priori)

  def _fit_params(self, X, y):
    # estimate all needed parameters for given model
    raise NotImplementedError()

  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    raise NotImplementedError()

  def fit(self, X, y, a_priori=None):
    # if it's needed, estimate a priori probabilities
    self.log_a_priori = self._estimate_a_priori(y) if a_priori is None else np.log(a_priori)

    # now that everything else is in place, estimate all needed parameters for given model
    self._fit_params(X, y)
    # Q4: por que el _fit_params va al final? no se puede mover a, por ejemplo, antes de la priori?

  def predict(self, X):
    # this is actually an individual prediction encased in a for-loop
    m_obs = X.shape[1]
    y_hat = np.empty(m_obs, dtype=int)

    for i in range(m_obs):
      y_hat[i] = self._predict_one(X[:,i].reshape(-1,1))

    # return prediction as a row vector (matching y)
    return y_hat.reshape(1,-1)

  def _predict_one(self, x):
    # calculate all log posteriori probabilities (actually, +C)
    log_posteriori = [ log_a_priori_i + self._predict_log_conditional(x, idx) for idx, log_a_priori_i
                  in enumerate(self.log_a_priori) ]

    # return the class that has maximum a posteriori probability
    return np.argmax(log_posteriori)

In [10]:
class QDA_Chol1(BaseBayesianClassifier):
  def _fit_params(self, X, y):
    self.L_invs = [
        LA.inv(cholesky(np.cov(X[:,y.flatten()==idx], bias=True), lower=True))
        for idx in range(len(self.log_a_priori))
    ]

    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]

  def _predict_log_conditional(self, x, class_idx):
    L_inv = self.L_invs[class_idx]
    unbiased_x =  x - self.means[class_idx]

    y = L_inv @ unbiased_x

    return np.log(L_inv.diagonal().prod()) -0.5 * (y**2).sum()


class QDA_Chol2(BaseBayesianClassifier):
  def _fit_params(self, X, y):
    self.Ls = [
        cholesky(np.cov(X[:,y.flatten()==idx], bias=True), lower=True)
        for idx in range(len(self.log_a_priori))
    ]

    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]

  def _predict_log_conditional(self, x, class_idx):
    L = self.Ls[class_idx]
    unbiased_x =  x - self.means[class_idx]

    y = solve_triangular(L, unbiased_x, lower=True)

    return -np.log(L.diagonal().prod()) -0.5 * (y**2).sum()


class QDA_Chol3(BaseBayesianClassifier):
  def _fit_params(self, X, y):
    self.L_invs = [
        dtrtri(cholesky(np.cov(X[:,y.flatten()==idx], bias=True), lower=True), lower=1)[0]
        for idx in range(len(self.log_a_priori))
    ]

    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]

  def _predict_log_conditional(self, x, class_idx):
    L_inv = self.L_invs[class_idx]
    unbiased_x =  x - self.means[class_idx]

    y = L_inv @ unbiased_x

    return np.log(L_inv.diagonal().prod()) -0.5 * (y**2).sum()

Las clases `QDA_Chol*` implementan, como su nombre lo indica, un algoritmo para aplicar Análisis Discriminante Cuadrático (QDA). Su diferencia está en cómo cada una realiza esta tarea. Todas se basan en la descomposición de Cholesky de la matriz de covarianianza ($\sigma$), que consiste en encontrar una matriz triangular inferior única $L$ tal que $\Sigma = L\cdot L^T$.

### QDA_Chol1

In [11]:
class QDA_Chol1(BaseBayesianClassifier):
  def _fit_params(self, X, y):
    self.L_invs = [
        LA.inv(cholesky(np.cov(X[:,y.flatten()==idx], bias=True), lower=True))
        for idx in range(len(self.log_a_priori))
    ]

    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]

  def _predict_log_conditional(self, x, class_idx):
    L_inv = self.L_invs[class_idx]
    unbiased_x =  x - self.means[class_idx]

    y = L_inv @ unbiased_x

    return np.log(L_inv.diagonal().prod()) -0.5 * (y**2).sum()

Esta es la implementación más *naive* de todas. Su punto a favor es que resulta fácil de escribir y entender. En contra, su ineficiencia computacional.

#### _fit_params

Comenzamos con el ajuste. Tiene dos atributos

- **Inversa:** Primero, se calcula la matriz de covarianza $\Sigma_k$​ para los datos de la clase actual, para luego aplicarle la descomposición de Cholesky sobre $\Sigma_k$ tal que $\Sigma_k = L_k L_k^T$. A esto, simplemente se le aplica su inversa (lo que computacionalmente suele ser relativamente demandante).

- **Vector de medias** El código itera sobre cada clase. Para cada una de ellas, selecciona de la matriz de datos $X$ solo las columnas que corresponden a esa clase. A esto, le toma la media (`axis=1` porque lo hace a través de las columnas).

#### _predict_log_conditional

A partir de `_fit_params`, obtiene vector de medias $\mu_k$ y la inversa de Cholesky $L_k^{-1}$ para calcular $y=L_k^{-1}\cdot (x-\mu_k)$. El último paso es aplicar la fórmula para la log probabilidad Normal multivariante.

### QDA_Chol2

Acá nos evitamos directamente el cálculo de la inversa y resolvemos el sistema lineal directamente usando `solve_triangular`.

In [13]:
class QDA_Chol2(BaseBayesianClassifier):
  def _fit_params(self, X, y):
    self.Ls = [
        cholesky(np.cov(X[:,y.flatten()==idx], bias=True), lower=True)
        for idx in range(len(self.log_a_priori))
    ]

    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]

  def _predict_log_conditional(self, x, class_idx):
    L = self.Ls[class_idx]
    unbiased_x =  x - self.means[class_idx]

    y = solve_triangular(L, unbiased_x, lower=True)

    return -np.log(L.diagonal().prod()) -0.5 * (y**2).sum()

#### _fit_params

Comenzamos con el ajuste. Tiene dos atributos:

- **Inversa:** Calcula la matriz de covarianza $\Sigma_k$​ para la clase actual para luego aplicar Cholesky sobre $\Sigma_k$ para obtener la matriz triangular inferior $L_k$.

- **Vector de medias** Igual a QDA_Chol1.

#### _predict_log_conditional

A partir de que la ecuación $y=L_k^ {−1}​(x−\mu_k​)$ es matemáticamente idéntica a resolver el sistema de ecuaciones lineales $L_k​y=(x−\mu_k​)$ para la incógnita $y$, se resuelve el sistema con la función solve_triangular para luego calcular la log-probabilidad, con la diferencia de que ahora se toma el logaritmo del producto de la diagonal de la matriz L, resultando algo más directo que QDA_Chol1. Al evitar el cálculo de la inversa, esta implementación resulta la **más eficiente computacionalmente**.

### QDA_Chol3

In [ ]:
class QDA_Chol3(BaseBayesianClassifier):
  def _fit_params(self, X, y):
    self.L_invs = [
        dtrtri(cholesky(np.cov(X[:,y.flatten()==idx], bias=True), lower=True), lower=1)[0]
        for idx in range(len(self.log_a_priori))
    ]

    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]

  def _predict_log_conditional(self, x, class_idx):
    L_inv = self.L_invs[class_idx]
    unbiased_x =  x - self.means[class_idx]

    y = L_inv @ unbiased_x

    return np.log(L_inv.diagonal().prod()) -0.5 * (y**2).sum()

Este es un punto intermedio, que no evita calcular la inversa, pero lo hace de un modo ligeramente más eficiente que QDA_Chol1.

#### _fit_params

Comenzamos con el ajuste. Tiene dos atributos:

- **Inversa:** Calcula la matriz de covarianza $\Sigma_k$​ para la clase actual
para luego aplicar Cholesky sobre $\Sigma_k$ para obtener la matriz triangular
inferior $L_k$. Aquí, en lugar de usar la función genérica LA.inv(), utiliza
dtrtri. Esta es una función de bajo nivel proveniente de la biblioteca LAPACK.
Significa "double precision triangular inverse", y está optimizada para invertir
matrices triangulares.

- **Vector de medias** Igual a QDA_Chol1.

#### _predict_log_conditional

Idéntico a QDA_Chol1, simplemente porque ambos métodos trabajan con la inversa.